In [10]:
import numpy as np
from PIL import Image
from PIL import ImageStat
import cv2
import os
import math
import pandas as pd

In [11]:
pic_dir = "./bmw10_ims/1/"
spec_folder=pic_dir[-3:-1]
if spec_folder[0]=='/':
    spec_folder='0'+spec_folder[1]
pics=os.listdir(pic_dir)
print(len(pics))
print(pics)
spec_folder

53
['150083719.jpg', '150086217.jpg', '150084051.jpg', '150105852.jpg', '150086093.jpg', '150103548.jpg', '150101815.jpg', '150101084.jpg', '150100259.thumb.jpg', '150081589.thumb.jpg', '150085397.thumb.jpg', '150100437.thumb.jpg', '150100905.jpg', '150105059.jpg', '150102427.jpg', '150085649.thumb.jpg', '150100755.thumb.jpg', '150101560.thumb.jpg', '150105100.jpg', '150084056.jpg', '150083780.jpg', '150082359.jpg', '150085363.thumb.jpg', '150082992.jpg', '150102021.jpg', '150081888.jpg', '150103629.jpg', '150103428.jpg', '150086282.jpg', '150080038.jpg', '150083647.jpg', '150083324.jpg', '150081573.thumb.jpg', '150101960.jpg', '150102991.jpg', '150084991.jpg', '150082962.jpg', '150101196.jpg', '150085495.jpg', '150105462.jpg', '150080476.jpg', '150081110.jpg', '150085089.jpg', '150080490.jpg', '150080705.jpg', '150084171.jpg', '150079887.jpg', '150084952.jpg', '150084715.thumb.jpg', '150101015.jpg', '150085542.jpg', '150083295.jpg', '150080973.thumb.jpg']


'01'

In [12]:
image_name=[]
clarity=[]
Brightness=[]
Pixel=[]
Contrast=[]
Resolution=[]
Vignette=[]
for pic in pics:
    
    image_name.append(pic)
    
    x=cv2.imread(pic_dir+pic)
    gray = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    clarity.append(fm)
    
    b=x[:,:,0]
    g=x[:,:,1]
    r=x[:,:,2]
    r=np.array(r,np.float32)
    g=np.array(g,np.float32)
    b=np.array(b,np.float32)
    gs = (np.sqrt(.241*(r**2)+.691*(g**2)+.068*(b**2)))
    avg = np.average(gs)
    Brightness.append(avg)
    
    dp=math.sqrt(x.shape[0]**2+x.shape[1]**2)
    Pixel.append(dp/5)
    
    hist = cv2.calcHist([x], [0], None, [256], [0, 256])
    e=sum(hist*np.log2(hist))
    Contrast.append(e[0])
    
    r = min(x.shape[0],x.shape[1])
    Resolution.append(r)
    
    rows, cols = x.shape[:2]
    kernel_x = cv2.getGaussianKernel(cols,200)
    kernel_y = cv2.getGaussianKernel(rows,200)
    kernel = kernel_y*kernel_x.T
    mask = 255 * kernel / np.linalg.norm(kernel)
    v=np.amax(mask)
    Vignette.append(v)
    


print(len(image_name))
print(len(clarity))
print(len(Brightness))
print(len(Pixel))
print(len(Contrast))
print(len(Resolution))
print(len(Vignette))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply


53
53
53
53
53
53
53


In [13]:
df=pd.DataFrame({'image_name':image_name,'clarity': clarity,
           'Brightness': Brightness,'Pixel':Pixel,'Contrast':Contrast,'Resolution':Resolution,'Vignette':Vignette})
#df.to_csv("data.csv")

In [14]:
df = df.dropna(axis=0)
df=df.reset_index(drop=True)
filename="data"+spec_folder+".csv"
df.to_csv(filename)